In [168]:
import numpy as np
import pandas as pd

data = pd.read_csv("../data/datos_grasas_Tec.csv", encoding="latin1")

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 41 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   idDatosGrasas                             51 non-null     int64  
 1   codigoGrasa                               51 non-null     object 
 2   Aceite Base                               47 non-null     object 
 3   Espesante                                 45 non-null     object 
 4   Grado NLGI Consistencia                   51 non-null     float64
 5   Viscosidad del Aceite Base a 40°C. cSt    50 non-null     float64
 6   Penetración de Cono a 25°C, 0.1mm         51 non-null     object 
 7   Punto de Gota, °C                         51 non-null     object 
 8   Estabilidad Mecánica, %                   48 non-null     object 
 9   Punto de Soldadura Cuatro Bolas, kgf      50 non-null     float64
 10  Desgaste Cuatro Bolas, mm               

In [169]:
def arreglar_rangos(data, columna):
    col1 = []
    col2 = []
    
    for i in data[columna]:
        col1.append(i[:3])
        col2.append(i[-3:])
    
    data.insert(loc=len(data.columns), column=columna+" min", value=col1)
    data.insert(loc=len(data.columns), column=columna+" max", value=col2)
    
    data[columna+" min"] = pd.to_numeric(data[columna+" min"])
    data[columna+" max"] = pd.to_numeric(data[columna+" max"])
    
    data.drop(columns=[columna], inplace=True)

arreglar_rangos(data, "Penetración de Cono a 25°C, 0.1mm")

In [170]:
def arreglar_desigualdades(data, columna, numero_rangos=5):
    filtro = pd.to_numeric(data[columna], errors='coerce').notna()
    
    data_numeros = data[filtro]
    data_objetos = data[~filtro]

    numeros = data_numeros[columna].unique().tolist()
    objetos = data_objetos[columna].unique().tolist()
    
    for i in objetos:
        i_str = str(i)
        if i_str[:1] == "<" or i_str[:1] == ">":
            numeros.append(pd.to_numeric(i_str[1:]))

    rangos = np.linspace(min(pd.to_numeric(numeros)), max(pd.to_numeric(numeros)), numero_rangos - 1)
    rangos = np.linspace(min(rangos) - (rangos[1] - rangos[0]), max(rangos) + (rangos[1] - rangos[0]), numero_rangos + 1)
    
    columnas = {}
    
    for i in range(numero_rangos):
        columnas[f"{rangos[i]} - {rangos[i+1]}"] = []
        
    if "No Gotea" in objetos:
        No_Gotea = []
    
    for i in data[columna].tolist():
        for j in range(numero_rangos):
            i_str = str(i)
            if (not pd.isna(pd.to_numeric(i_str, errors='coerce'))) and (rangos[j] < pd.to_numeric(i_str) <= rangos[j+1]):
                columnas[f"{rangos[j]} - {rangos[j+1]}"].append(1)
            elif (i_str[:1] == "<") and (pd.to_numeric(i_str[1:]) > rangos[j+1]):
                columnas[f"{rangos[j]} - {rangos[j+1]}"].append(1)
            elif (i_str[:1] == ">") and (pd.to_numeric(i_str[1:]) < rangos[j]):
                columnas[f"{rangos[j]} - {rangos[j+1]}"].append(1)
            else:
                columnas[f"{rangos[j]} - {rangos[j+1]}"].append(0)
                
        if ("No Gotea" in objetos) and (i == "No Gotea"):
            No_Gotea.append(1)
        elif ("No Gotea" in objetos) and (i != "No Gotea"):
            No_Gotea.append(0)
    
    for i in range(numero_rangos):
        data.insert(loc=len(data.columns), column=columna+f" {rangos[i]:.2f} - {rangos[i+1]:.2f}", value=columnas[f"{rangos[i]} - {rangos[i+1]}"])
        data[columna+f" {rangos[i]:.2f} - {rangos[i+1]:.2f}"] = pd.to_numeric(data[columna+f" {rangos[i]:.2f} - {rangos[i+1]:.2f}"])
        
    if "No Gotea" in objetos:
        data.insert(loc=len(data.columns), column=columna+" No Gotea", value=No_Gotea)
        data[columna+" No Gotea"] = pd.to_numeric(data[columna+" No Gotea"])
        
    data.drop(columns=[columna], inplace=True)

arreglar_desigualdades(data, "Punto de Gota, °C")
arreglar_desigualdades(data, "Estabilidad Mecánica, %")
arreglar_desigualdades(data, "Carga Timken Ok, lb")
arreglar_desigualdades(data, "Resistencia al Lavado por Agua a 80°C, %")

In [171]:
def codificador(data, columna):
    columnas_nuevas = pd.get_dummies(data[columna], prefix=columna)
    columnas_nuevas = columnas_nuevas.astype(int)
    
    data[columnas_nuevas.columns] = columnas_nuevas
    data.drop(columns=[columna], inplace=True)

codificador(data, "Aceite Base")
codificador(data, "Espesante")
codificador(data, "Clasificacion ISO 6743-9")
codificador(data, "color")
codificador(data, "textura")

In [172]:
# eliminar columnas tipo objeto con pocas observaciones
data.drop(columns=["codigoGrasa",
                   "Corrion EMCOR",
                   "Corrosión al Cobre",
                   "Bombeabilidad",
                   "Separacion del Aceite, %",
                   "Prevencion a la Corrosion",
                   "Resistencia a la Humedad",
                   "categoria"], inplace=True)

# eliminar columnas tipo numerico con pocas observaciones
data.drop(columns=["Indice de Carga-Desgaste",
                   "Registro NSF",
                   "Presion de Flujo a -30°C, mbar"], inplace=True)

# eliminar columnas 0 non-null
data.drop(columns=["Prueba FZG",
                   "Solidos Totales, %",
                   "Tamaño de Particula, µ",
                   "Estabilidad a la Oxidacion",
                   "pH",
                   "Factor de Torque",
                   "Viscosidad Dinámica a 25°C, cP",
                   "Biodegradabilidad en 28 dias (> 60%)"], inplace=True)

In [173]:
for col in data.columns:
    nulos = data[col].notnull().sum()
    filas = len(data)
    dtype = data[col].dtype

    if nulos != filas and dtype != 'object':
        data[col] = data[col].fillna(0)
        print(f"{dtype}, {col}")

float64, Viscosidad del Aceite Base a 40°C. cSt
float64, Punto de Soldadura Cuatro Bolas, kgf
float64, Desgaste Cuatro Bolas, mm
float64, Factor de Velocidad


In [174]:
no_numericas = []

for i in data.columns:
    if data[i].dtype not in ['int64', 'float64', 'int32', 'float32']:
        no_numericas.append(i)
        print()
        print(len(data[i]))
        print(10*"=")
        print(f"{i}: {data[i].dtype}, {data[i].nunique()}")
        print(10*"=")
        for j in data[i].unique():
            print(j)

print()
print(no_numericas)


51
subtitulo: object, 19
Grasa de servicio pesado para alta resistencia al lavado por agua, temperatura y larga duraciÃ³n. 
Grasa para lubricaciÃ³n de equipo pesado.
Grasa lubricante para condiciones extremas.
Grasa lubricante para servicio pesado con alta resistencia a humedad y alto soporte a cargas extremas.
Grasa lubricante especial para cables.
Grasa de servicio pesado para alta temperatura, larga duraciÃ³n y resistente al lavado por agua.
Grasa lubricante sellante para vÃ¡lvulas de servicio en larga duraciÃ³n resistentes a gas natural y L.P.
Grasa de alta resistencia para industria alimenticia y farmacÃ©utica.
Grasa lubricante para rodamientos de peletizadoras de madera.
Grasa para alta temperatura y larga duraciÃ³n.
Grasa adherente para mecanismos automotrices.
Grasa especial para la industria alimenticia y farmacÃ©utica.
Lubricante especial para engranes de palas mecÃ¡nicas.
Grasa lubricante con alta resistencia a humedad y medios quÃ­micos con alto soporte a cargas extremas.


In [175]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 63 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   idDatosGrasas                                          51 non-null     int64  
 1   Grado NLGI Consistencia                                51 non-null     float64
 2   Viscosidad del Aceite Base a 40°C. cSt                 51 non-null     float64
 3   Punto de Soldadura Cuatro Bolas, kgf                   51 non-null     float64
 4   Desgaste Cuatro Bolas, mm                              51 non-null     float64
 5   Factor de Velocidad                                    51 non-null     float64
 6   Temperatura de Servicio °C, min                        51 non-null     int64  
 7   Temperatura de Servicio °C, max                        51 non-null     int64  
 8   subtitulo                                           

# Ignorar

In [176]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Los vectores deben estar en forma de array de NumPy y con la forma correcta
vector_a = np.array([[1, 2, 3]])
vector_b = np.array([[4, 5, 6]])

similitud = cosine_similarity(vector_a, vector_b)
print(f"La similitud del coseno es: {similitud[0][0]}")

La similitud del coseno es: 0.9746318461970762


In [177]:
from sklearn.metrics.pairwise import cosine_similarity
X = [[0, 0, 0], [1, 1, 1]]
Y = [[1, 0, 0], [1, 1, 0]]
cosine_similarity(X, Y)

array([[0.        , 0.        ],
       [0.57735027, 0.81649658]])